In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import layers
import pandas as pd
import os
import matplotlib.pyplot as plt
import copy
os.environ['CUDA_VISIBLE_DEVICES']="0"
pd_train=pd.read_csv("../../data/train.csv")
pd_test=pd.read_csv("../../data/test.csv")
pd_submission=pd.read_csv("../../data/sample_submission.csv")

In [ ]:
def DatasetComposition(pddata,test_pddata,organize,test):
    train_ID=np.array(pddata["ID"])
    pddata["주행거리"]=pddata["주행거리"]/10000
    brand=pd.unique(pddata["브랜드"])
    brand_record=np.copy(brand)
    pddata=pddata.drop(["판매도시"],axis=1)
    pddata=pddata.drop(["판매구역"],axis=1)
    vehicle_model=pd.unique(pddata["차량모델명"])
    vehicle_model_record=np.copy(vehicle_model)
    if organize=="min":
        for i in range(len(brand)):
            brand_record[i]=pddata.loc[pddata["브랜드"]==brand[i]]["가격"].min()
        for i in range(len(vehicle_model)):
            vehicle_model_record[i]=pddata.loc[pddata["차량모델명"]==vehicle_model[i]]["가격"].min()
    elif organize=="mean":
        for i in range(len(brand)):
            brand_record[i]=pddata.loc[pddata["브랜드"]==brand[i]]["가격"].mean()
        for i in range(len(vehicle_model)):
            vehicle_model_record[i]=pddata.loc[pddata["차량모델명"]==vehicle_model[i]]["가격"].mean()
    elif organize=="max":
        for i in range(len(brand)):
            brand_record[i]=pddata.loc[pddata["브랜드"]==brand[i]]["가격"].max()
        for i in range(len(vehicle_model)):
            vehicle_model_record[i]=pddata.loc[pddata["차량모델명"]==vehicle_model[i]]["가격"].max()
 
    production_year_max=pddata["생산년도"].max()
    production_year_min=pddata["생산년도"].min()
    model_release_year_max=pddata["모델출시년도"].max()
    model_release_year_min=pddata["모델출시년도"].min()
    distance_driven_max=pddata["주행거리"].max()
    distance_driven_min=pddata["주행거리"].min()
    Displacement_max=pddata["배기량"].max()
    Displacement_min=pddata["배기량"].min()
 
    pddata["생산년도"]=(pddata["생산년도"]-production_year_min)/(production_year_max-production_year_min)
    pddata["모델출시년도"]=(pddata["모델출시년도"]-model_release_year_min)/(model_release_year_max-model_release_year_min)
    pddata["주행거리"]=(pddata["주행거리"]-distance_driven_min)/(distance_driven_max-distance_driven_min)
    pddata["배기량"]=(pddata["배기량"]-Displacement_min)/(Displacement_max-Displacement_min)

    pddata.insert(3,'생산-출시',pddata["생산년도"]-pddata["모델출시년도"])
    pddata=pddata.drop(["모델출시년도"],axis=1)
    np_tarin=np.array(pddata)
    for i in range(len(np_tarin)):
        np_tarin[i,3]=brand_record[np.where(brand==np_tarin[i,3])[0][0]]/brand_record.max()
        np_tarin[i,4]=vehicle_model_record[np.where(vehicle_model==np_tarin[i,4])[0][0]]/vehicle_model_record.max()
    np_tarin=np.delete(np_tarin,0,axis=1)
    
    if test:
        test_pddata["주행거리"]=test_pddata["주행거리"]/10000
        test_pddata=test_pddata.drop(["판매도시"],axis=1)
        test_pddata=test_pddata.drop(["판매구역"],axis=1)
        test_pddata["생산년도"]=(test_pddata["생산년도"]-production_year_min)/(production_year_max-production_year_min)
        test_pddata["모델출시년도"]=(test_pddata["모델출시년도"]-model_release_year_min)/(model_release_year_max-model_release_year_min)
        test_pddata["주행거리"]=(test_pddata["주행거리"]-distance_driven_min)/(distance_driven_max-distance_driven_min)
        test_pddata["배기량"]=(test_pddata["배기량"]-Displacement_min)/(Displacement_max-Displacement_min)
        test_pddata.insert(3,'생산-출시',test_pddata["생산년도"]-test_pddata["모델출시년도"])
        test_pddata=test_pddata.drop(["모델출시년도"],axis=1)
        np_test=np.array(test_pddata)
        for i in range(len(np_test)):
            np_test[i,3]=brand_record[np.where(brand==np_test[i,3])[0][0]]/brand_record.max()
            np_test[i,4]=vehicle_model_record[np.where(vehicle_model==np_test[i,4])[0][0]]/vehicle_model_record.max()
        np_test=np.delete(np_test,0,axis=1) 
        return np_test
    
    return np_tarin[:,:-1],np_tarin[:,-1]

In [101]:
pd_train1=copy.copy(pd_train)
mean_train_x,mean_train_y=DatasetComposition(pd_train1,'',"mean",False)
mean_train_x=mean_train_x.astype(np.float32)
mean_train_y=mean_train_y.astype(np.float32)
pd_train1=copy.copy(pd_train)
min_train_x,min_train_y=DatasetComposition(pd_train1,'',"min",False)
min_train_x=min_train_x.astype(np.float32)
min_train_y=min_train_y.astype(np.float32)
pd_train1=copy.copy(pd_train)
max_train_x,max_train_y=DatasetComposition(pd_train1,'',"max",False)
max_train_x=max_train_x.astype(np.float32)
max_train_y=max_train_y.astype(np.float32)
pd_train1=copy.copy(pd_train)
pd_test1=copy.copy(pd_test)
test_x=DatasetComposition(pd_train1,pd_test1,"max",True)
test_x=test_x.astype(np.float32)

In [102]:

def lstm_model():
    model = K.Sequential()
    model.add(layers.LSTM(44, input_shape = (11,1), return_sequences = True))
    model.add(layers.LSTM(22,  return_sequences = True))
    model.add(layers.LSTM(11,  return_sequences = False))
    model.add(layers.Dense(1))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam')
    return model

def gru_model():
    model = K.Sequential()
    model.add(layers.GRU(44, input_shape = (11,1), return_sequences = True))
    model.add(layers.GRU(22,  return_sequences = True))
    model.add(layers.GRU(13, return_sequences = False))
    model.add(layers.Dense(1))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam')
    return model

lstm_Model = lstm_model()
gru_Model = gru_model()

In [103]:
mc = K.callbacks.ModelCheckpoint("../../model/gru/mean/bast_mse.h5", save_best_only=True)
gru_Model.fit(mean_train_x[:-10000],mean_train_y[:-10000],validation_data=(mean_train_x[-10000:],mean_train_y[-10000:]),callbacks=[mc], batch_size = 100, epochs = 2000, verbose = 1)
gru_Model.save("../../model/gru/mean/mse.h5")

Epoch 1/2000
480/480 [==============================] - 72s 89ms/step - loss: 43.9908 - val_loss: 40.4341
Epoch 2/2000
480/480 [==============================] - 34s 71ms/step - loss: 37.9690 - val_loss: 35.5430
Epoch 3/2000
480/480 [==============================] - 33s 70ms/step - loss: 33.9206 - val_loss: 32.2504
Epoch 4/2000
480/480 [==============================] - 32s 68ms/step - loss: 31.3293 - val_loss: 30.2956
Epoch 5/2000
480/480 [==============================] - 34s 72ms/step - loss: 29.7854 - val_loss: 29.1105
Epoch 6/2000
480/480 [==============================] - 31s 65ms/step - loss: 28.8863 - val_loss: 28.4695
Epoch 7/2000
480/480 [==============================] - 31s 64ms/step - loss: 28.4057 - val_loss: 28.1337
Epoch 8/2000
480/480 [==============================] - 32s 66ms/step - loss: 28.1598 - val_loss: 27.9621
Epoch 9/2000
480/480 [==============================] - 23s 49ms/step - loss: 28.0655 - val_loss: 27.9094
Epoch 10/2000
480/480 [=======================

In [ ]:
mc = K.callbacks.ModelCheckpoint("../../model/lstm/mean/bast_mse.h5", save_best_only=True)
lstm_Model.fit(mean_train_x[:-10000],mean_train_y[:-10000],validation_data=(mean_train_x[-10000:],mean_train_y[-10000:]),callbacks=[mc], batch_size = 100, epochs = 2000, verbose = 1)
lstm_Model.save("../../model/lstm/mean/mse.h5")

In [ ]:
pred_x=gru_Model.predict(test_x)

In [ ]:
pd_submission1=copy.copy(pd_submission)
for i in range(len(pred_x)):
    pd_submission1.loc[i,"가격"]=pred_x[i]

In [ ]:
pd_submission1.to_csv('../../data/mean_gru_submission.csv', index = None)